Data Loading, ## EDA

In [0]:
%pip install pandas matplotlib seaborn numpy


In [0]:
%restart_python


Import Libraries & Load Data

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ast # For safe list parsing in genres/countries

tpath="./titles.csv"
cpath="./credits.csv"

titledf = pd.read_csv(tpath)
creaditsdf = pd.read_csv(cpath)

 
 
display(titledf)
display(creaditsdf)
print("Data loaded successfully!")
print(f"Titles shape: {titledf.shape}")
print(f"Credits shape: {creaditsdf.shape}")

Data Exploration (Head, Tail, Summary, Data Dictionary)


In [0]:
# 1. Head and Tail
print("=== TITLES DATA HEAD ===")
print(titledf.head())



In [0]:
# print("=== TITLES DATA TAIL ===")
titledf.tail()

In [0]:
# print("=== CREDITS DATA HEAD ===")
print( creaditsdf.head())
 

In [0]:
# print("=== CREDITS DATA TAIL ===")
print(  creaditsdf.tail())

In [0]:
# 2. Summary Statistics
print("\n=== TITLES SUMMARY ===")
print(titledf.describe(include='all'))  # All columns: numeric + categorical

print("\n=== CREDITS SUMMARY ===")
print(creaditsdf.describe(include='all'))

In [0]:
# 3. Data Dictionary (Manual based on description)
data_dict = {
    'titles.csv': {
        'id': 'Title ID on JustWatch (str)',
        'title': 'Name of the title (str)',
        'type': 'MOVIE or SHOW (str)',
        'description': 'Brief description (str)',
        'release_year': 'Release year (int)',
        'age_certification': 'Age rating (str)',
        'runtime': 'Length in minutes (int/float)',
        'genres': 'List of genres (str, e.g., "["comedy"]")',
        'production_countries': 'List of countries (str, e.g., "["US"]")',
        'seasons': 'Number of seasons (float/int, NaN for movies)',
        'imdb_id': 'IMDB ID (str)',
        'imdb_score': 'IMDB rating (float)',
        'imdb_votes': 'IMDB votes (float)',
        'tmdb_popularity': 'TMDB popularity (float)',
        'tmdb_score': 'TMDB score (float)'
    },
    'credits.csv': {
        'person_id': 'Person ID on JustWatch (int)',
        'id': 'Title ID (str)',
        'name': "Actor/Director's name (str)",
        'character': 'Character name (str)',
        'role': 'ACTOR or DIRECTOR (str)'
    }
}

import pandas as pd

dict_df = pd.DataFrame(
    [(dataset, col_name, desc) for dataset, columns in data_dict.items() for col_name, desc in columns.items()],
    columns=['Dataset', 'Column', 'Description']
)
display(dict_df)

display(titledf.dtypes)
display(creaditsdf.dtypes)
 

In [0]:
# 2. Handling Strategy (Logical & Efficient)
# Titles: Drop rows with missing title/type (core).
#  Fill release_year with median.
#  Drop high-missing like seasons (movie-specific).
 

In [0]:
# Check types in 'genres' (and 'production_countries')
print("=== TYPES IN GENRES ===")

# print("\n=== SAMPLE VALUES (First 5) ===")
# for i, val in enumerate(titledf['genres'].head()):
#     print(f"Row {i}: Type={type(val)}, Value={val}")

print("\n=== NaNs? ===")
print("Genres NaNs:", titledf['genres'].isna().sum())
print(" production_countries NaNs:", titledf['production_countries'].isna().sum())


In [0]:
# Fill Missing values
titledf['release_year']= titledf['release_year'].fillna(titledf['release_year'].median())

display(titledf)

In [0]:
 

titledf['age_certification'] = titledf['age_certification'].fillna('Unknown')
display(titledf)




In [0]:
titledf['seasons']= titledf['seasons'].fillna(1)


In [0]:
tmdb_median = titledf['tmdb_score'].median()
display(tmdb_median)
titledf['tmdb_score']=titledf['tmdb_score'].fillna(tmdb_median)


In [0]:
imdbvotes=titledf['imdb_votes'].median()
display(imdbvotes)
titledf['imdb_votes']=titledf['imdb_votes'].fillna(imdbvotes)


In [0]:
imdbScore=titledf['imdb_score'].median()
display(imdbScore)
titledf['imdb_score']=titledf['imdb_score'].fillna(imdbScore)



In [0]:
imdid='Unknown'
titledf['imdb_id']=titledf['imdb_id'].fillna(imdid)

In [0]:
tmdb_pop_median = titledf['tmdb_popularity'].median()
titledf['tmdb_popularity'] = titledf['tmdb_popularity'].fillna(tmdb_pop_median)

In [0]:
titledf['description'] = titledf['description'].fillna('No description available.')

In [0]:
# 1. Missing Values Analysis
print("=== MISSING VALUES (TITLES) ===")
# titles_pd = titledf
credits_pd = creaditsdf
titles_missing = titledf.isnull().sum().to_frame(name='Count').sort_values('Count', ascending=False)
titles_missing['Percentage'] = round((titles_missing['Count'] / len(titledf)) * 100, 2)
print(titles_missing[titles_missing['Count'] > 0])

# Show sample data for 'production_countries' and 'genres'
# print("\n=== SAMPLE DATA: production_countries & genres ===")
# print(titledf[['production_countries', 'genres']].head(10))




In [0]:
display(titledf)

In [0]:
print("\n=== MISSING VALUES (CREDITS) ===")
credits_missing = credits_pd.isnull().sum().to_frame(name='Count').sort_values('Count', ascending=False)
credits_missing['Percentage'] = round((credits_missing['Count'] / len(credits_pd)) * 100, 2)
print(credits_missing[credits_missing['Count'] > 0])

In [0]:
creaditsdf.loc[(creaditsdf['role'] == 'DIRECTOR') & (creaditsdf['character'].isnull()), 'character'] = 'DIRECTOR'
mask_actor = (creaditsdf['role'] == 'ACTOR') & creaditsdf['character'].isna()
creaditsdf.loc[mask_actor, 'character'] = 'Unknown Character'
 

In [0]:
# Merge datasets on 'id' for joint analysis (left join to keep all titles)
merged_pd = pd.merge(titledf, credits_pd[['id', 'name', 'role']], on='id', how='left')

# Milestone 1: Content Diversity (Genres & Types)
# Explode genres for counting (handle multi-genre titles)
exploded_genres = titledf.explode('genres')
genre_counts = exploded_genres['genres'].value_counts().head(10)
print("=== TOP 10 GENRES (Diversity) ===")
print(genre_counts)

type_dist = titledf['type'].value_counts()
print("\n=== CONTENT TYPES ===")
print(type_dist)

# Milestone 2: Regional Availability (Production Countries)
top_countries = titledf.explode('production_countries')['production_countries'].value_counts().head(10)
print("\n=== TOP 10 PRODUCTION COUNTRIES ===")
print(top_countries)

# Milestone 3: Trends Over Time (Release Year)
titledf['decade'] = (titledf['release_year'] // 10) * 10
year_trend = titledf.groupby('decade').size().reset_index(name='Count')
print("\n=== CONTENT BY DECADE (Trends) ===")
print(year_trend)

# Milestone 4: IMDb Ratings & Popularity
top_rated = titledf.nlargest(10, 'imdb_score')[['title', 'type', 'imdb_score', 'imdb_votes', 'tmdb_popularity']]
print("\n=== TOP 10 RATED TITLES ===")
print(top_rated)

# Correlations (Numeric: scores, popularity, votes, runtime)
numeric_cols = ['imdb_score', 'tmdb_score', 'tmdb_popularity', 'imdb_votes', 'runtime']
corr_matrix = titledf[numeric_cols].corr()
print("\n=== CORRELATION MATRIX ===")
print(corr_matrix.round(2))

# Quick Insights (Automated)
print("\n=== KEY TRENDS ===")
if corr_matrix.loc['imdb_votes', 'imdb_score'] > 0.5:
    print("- Strong positive correlation: More votes → Higher IMDB scores (popularity drives ratings).")
print(f"- Recent surge: {year_trend[year_trend['decade'] >= 2010]['Count'].sum() / len(titledf) * 100:.1f}% of library from 2010s+.")
print("- Diversity: Movies dominate; US-centric production suggests regional focus.")

In [0]:
display(creaditsdf)

In [0]:
# Set style for consistent plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Create subplots for 6 viz types
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Amazon Prime EDA: Key Visualizations', fontsize=16, y=0.98)

# Viz 1: Bar Plot - Top Genres (Categorical count)
genre_counts.plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Top 10 Genres (Diversity)')
axes[0,0].set_ylabel('Count')
axes[0,0].tick_params(axis='x', rotation=45)

# Viz 2: Histogram - Release Years (Numeric distribution)
titledf['release_year'].hist(bins=30, ax=axes[0,1], color='lightgreen', edgecolor='black')
axes[0,1].set_title('Release Year Distribution')
axes[0,1].set_xlabel('Year')

# Viz 3: Pie Chart - Content Types (Proportions)
type_dist.plot(kind='pie', autopct='%1.1f%%', ax=axes[0,2], startangle=90)
axes[0,2].set_title('Movies vs. Shows')

# Viz 4: Scatter Plot - IMDB Score vs. Votes (Correlation trend)
sns.scatterplot(data=titledf, x='imdb_votes', y='imdb_score', ax=axes[1,0], alpha=0.6)
axes[1,0].set_title('IMDB Score vs. Votes')
axes[1,0].set_xscale('log')  # Log scale for votes skew

# Viz 5: Box Plot - Runtime by Type (Spread & outliers)
sns.boxplot(data=titledf, x='type', y='runtime', ax=axes[1,1])
axes[1,1].set_title('Runtime by Content Type')

# Viz 6: Line Plot - Content Trend Over Decades (Time series)
year_trend.plot(x='decade', y='Count', kind='line', ax=axes[1,2], marker='o', color='red')
axes[1,2].set_title('Titles Added by Decade')
axes[1,2].set_xlabel('Decade')
axes[1,2].set_ylabel('Count')

plt.tight_layout()
plt.show()

#  Save to FileStore
plt.savefig('/Workspace/Users/rsangramofficial@gmail.com/EDA/Amazon Prime Video EDA Project/eda_viz.png', dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
print("Visualizations generated and saved!")